In [126]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as soup
import lxml
from bs4 import NavigableString
import re

In [127]:
# Must gather rating data for all the games without data in the ratings_df in the exploratory analysis notebook.
# Must gather positive, neutral, and negative ratings numbers for both users and critics as well as the
# metascore and userscore

In [128]:
no_rating_df = pd.read_csv("../Resources/games_to_get_ratings.csv")

In [129]:
!which chromedriver

/usr/local/bin/chromedriver


In [176]:
browser = Browser("chrome", executable_path="/usr/local/bin/chromedriver", headless=True)

In [131]:
games_df = pd.read_csv("../Resources/vgsales.csv.zip")
list_of_games = games_df["Name"].values.tolist()

In [17]:
# Grabbing the Metascore

game_scores = {}
problem_games = []

for game in ["Super Smash Bros."]:
    browser.visit("https://www.metacritic.com/")
    # fill in my search term to the Metacritic Browser and us "\n" to press "enter" and initialize search
    browser.fill("search_term", game + "\n")
    html= browser.html
    my_soup = soup(html, "lxml")
    try:
        metascore = my_soup.find("a", class_="metascore_anchor").get_text()
        game_scores[game]= metascore
        print("SUCCESSFUL")
    except:
        problem_games.append(game)
        print("THIS IS A PROBLEM CHILD")

THIS IS A PROBLEM CHILD


In [132]:
my_df = pd.DataFrame(game_scores, columns=["Name","Metascore"])

In [133]:
## Navigating to the correct page via the search using splinter is too slow. I will rewrite the code to manually 
# write in the url. 
# I will need to interpolate the correct console name as well into the url. Metacritics url structure is:
# "https://www.metacritic.com/game/{console}/{game}"

In [134]:
# must change game names into the proper format to be interpolated into the URL.

game_conversions= {
    "PSV":"playstation-vita",
    "DC":"dreamcast",
    "WiiU":"wii-u",
    "GC":"gamecube",
    "N64":"nintendo-64",
    "XB":"xbox",
    "PSP":"psp",
    "PS4":"playstation-4",
    "PS": "playstation",
    "Wii":"wii",
    "PS3": "playstation-3",
    "PS2":"playstation-2",
    "GBA":"game-boy-advance",
    "DS":"ds",
    "XB":"xbox",
    "X360":"xbox-360"
}

def convert_to_name(platform_abb):
    if platform_abb in game_conversions:
        full_platform= game_conversions[platform_abb]
    else:
        full_platform="other"
    return full_platform
    

In [159]:
# This actually shows that the vast majority of our data belongs to the consoles listed on metacritic, these "legacy consoles"
# Get rid of those games which fall into the "other" category under full_platform_name
games_df["full_platform_name"]= games_df["Platform"].map(convert_to_name)
games_df = games_df.loc[games_df["full_platform_name"]!="other"]


In [115]:
# Grabbing the Metascore

game_scores = {}
problem_games = []


# zip the two 
zipped_list =list(zip(games_df["full_platform_name"].tolist(),games_df["Name"].tolist()))

for platform, name in zipped_list:
    edited_game_title= name.lower().replace(":","-").replace(" ","-").replace("\W|\s","-").replace("--","-")
    browser.visit(f"https://www.metacritic.com/game/{platform}/{edited_game_title}")
    # fill in my search term to the Metacritic Browser and us "\n" to press "enter" and initialize search
    
    html= browser.html
    my_soup = soup(html, "lxml")
    try:
        metascore = my_soup.find("a", class_="metascore_anchor").get_text()
        game_scores[game]= metascore
        print("SUCCESSFUL")
    except:
        problem_games.append(game)
        print("THIS IS A PROBLEM CHILD")

SUCCESSFUL
THIS IS A PROBLEM CHILD
SUCCESSFUL
SUCCESSFUL
THIS IS A PROBLEM CHILD
THIS IS A PROBLEM CHILD
THIS IS A PROBLEM CHILD
SUCCESSFUL
THIS IS A PROBLEM CHILD
THIS IS A PROBLEM CHILD
THIS IS A PROBLEM CHILD
SUCCESSFUL
THIS IS A PROBLEM CHILD
SUCCESSFUL
SUCCESSFUL
SUCCESSFUL
THIS IS A PROBLEM CHILD
SUCCESSFUL
THIS IS A PROBLEM CHILD
SUCCESSFUL
THIS IS A PROBLEM CHILD
THIS IS A PROBLEM CHILD
THIS IS A PROBLEM CHILD
SUCCESSFUL
SUCCESSFUL
THIS IS A PROBLEM CHILD
THIS IS A PROBLEM CHILD
SUCCESSFUL


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=86.0.4240.198)


In [21]:
# This worked pretty well, but I will alter the code further see if I can increase my scraping success to fail ratio

In [136]:
import re
mystring = "The Legend!of:Zelda; Breath of the wild"
x = re.compile("\W|\s")
mystring = re.sub(x,"-", mystring).lower().replace("--","-")
mystring

'the-legend-of-zelda-breath-of-the-wild'

In [161]:
# Grabbing the Metascore

game_scores = {}
problem_games = []
counter = 1

# zip the two 
zipped_list =list(zip(games_df["full_platform_name"].tolist(),games_df["Name"].tolist()))

for platform, name in zipped_list:
    x = re.compile("\W|\s")
    edited_game_title= re.sub(x,"-", name).lower().replace("--","-")
    # periods at the end of the url will show up as a dash. This conditional is to get rid of it.
    if edited_game_title[-1] == "-":
        edited_game_title = edited_game_title[:-1]
        print(edited_game_title)
    else:
        print(edited_game_title)
    browser.visit(f"https://www.metacritic.com/game/{platform}/{edited_game_title}")
    # fill in my search term to the Metacritic Browser and us "\n" to press "enter" and initialize search
    
    html= browser.html
    my_soup = soup(html, "lxml")
    try:
        metascore = my_soup.find("a", class_="metascore_anchor").get_text()
        game_scores[game]= metascore
        print("SUCCESSFUL")
        print(counter)
        counter+=1
    except:
        problem_games.append(game)
        print("THIS IS A PROBLEM CHILD")
        print(counter)
        counter+=1

wii-sports
SUCCESSFUL
1
mario-kart-wii
SUCCESSFUL
2
wii-sports-resort
SUCCESSFUL
3
new-super-mario-bros
SUCCESSFUL
4
wii-play
SUCCESSFUL
5
new-super-mario-bros-wii
SUCCESSFUL
6
nintendogs
THIS IS A PROBLEM CHILD
7
mario-kart-ds
SUCCESSFUL
8
wii-fit
SUCCESSFUL
9
wii-fit-plus
SUCCESSFUL
10
kinect-adventures
SUCCESSFUL
11
grand-theft-auto-v
SUCCESSFUL
12
grand-theft-auto-san-andreas
SUCCESSFUL
13
brain-age-train-your-brain-in-minutes-a-day
SUCCESSFUL
14
pokemon-diamond-pokemon-pearl
THIS IS A PROBLEM CHILD
15
grand-theft-auto-v
SUCCESSFUL
16
grand-theft-auto-vice-city
SUCCESSFUL
17
pokemon-ruby-pokemon-sapphire
THIS IS A PROBLEM CHILD
18
pokemon-black-pokemon-white
THIS IS A PROBLEM CHILD
19
brain-age-2-more-training-in-minutes-a-day
SUCCESSFUL
20
gran-turismo-3-a-spec
SUCCESSFUL
21
call-of-duty-modern-warfare-3
SUCCESSFUL
22
call-of-duty-black-ops
SUCCESSFUL
23
call-of-duty-black-ops-3
THIS IS A PROBLEM CHILD
24
call-of-duty-black-ops-ii
SUCCESSFUL
25
call-of-duty-black-ops-ii
SUCCESSFUL

KeyboardInterrupt: 

In [177]:
game_scores = {}
problem_games = []
game_info_list = []
counter = 1

# zip the two 
zipped_list =list(zip(games_df["full_platform_name"].tolist(),games_df["Name"].tolist()))

for platform, name in zipped_list:
    x = re.compile("\W|\s")
    edited_game_title= re.sub(x,"-", name).lower().replace("--","-")
    # periods at the end of the url will show up as a dash. This conditional is to get rid of it.
    if edited_game_title[-1] == "-":
        edited_game_title = edited_game_title[:-1]
        print(edited_game_title)
    else:
        print(edited_game_title)
    browser.visit(f"https://www.metacritic.com/game/{platform}/{edited_game_title}")
    # fill in my search term to the Metacritic Browser and us "\n" to press "enter" and initialize search
    
    game_scores["Name"]=name
    
    html= browser.html
    my_soup = soup(html, "lxml")
    # Grabbing the Metascore
    try:
        metascore = my_soup.find("a", class_="metascore_anchor").get_text()
        game_scores["metascore"]= metascore
    except:
        problem_games.append(name)
        print('THIS IS A PROBLEM CHILD')
        continue
    try:
        critic_reviews = []
        positive_scores = my_soup.find("div",class_="distribution_wrap")
        x = positive_scores.select("ol.score_counts.hover_none span.count")
        for i in x:
            critic_reviews.append(i.get_text())
        game_scores["positive_critic_reviews"] = critic_reviews[0]
        game_scores["mixed_critic_reviews"]=critic_reviews[1]
        game_scores["negative_critic_reviews"]=critic_reviews[2]
        game_info_list.append(game_scores)
        print('SUCCESSFUL')
    except:
        problem_games.append(name)
        print('THIS IS A PROBLEM CHILD')
        continue
    try:
        user_reviews = []
        user_scores = my_soup.find("div",class_="user_reviews_module")
        drilling_down = user_scores.find("div",class_="distribution_wrap")
        ordered_list_html = drilling_down.select("ol.score_counts.hover_none span.count")
        for i in ordered_list_html:
            user_reviews.append(i.get_text())
        game_scores["positive_user_reviews"] = user_reviews[0]
        game_scores["mixed_user_reviews"]=user_reviews[1]
        game_scores["negative_user_reviews"]=user_reviews[2]
    except:
        problem_games.append(name)
        print("THIS IS A PROBLEM")
        continue

        
        

wii-sports
SUCCESSFUL
mario-kart-wii
SUCCESSFUL
wii-sports-resort
SUCCESSFUL
new-super-mario-bros
SUCCESSFUL
wii-play
SUCCESSFUL
new-super-mario-bros-wii
SUCCESSFUL
nintendogs
THIS IS A PROBLEM CHILD
mario-kart-ds
SUCCESSFUL
wii-fit
SUCCESSFUL
wii-fit-plus
SUCCESSFUL
kinect-adventures
SUCCESSFUL
grand-theft-auto-v
SUCCESSFUL
grand-theft-auto-san-andreas
SUCCESSFUL
brain-age-train-your-brain-in-minutes-a-day
SUCCESSFUL
pokemon-diamond-pokemon-pearl
THIS IS A PROBLEM CHILD
grand-theft-auto-v
SUCCESSFUL
grand-theft-auto-vice-city
SUCCESSFUL
pokemon-ruby-pokemon-sapphire
THIS IS A PROBLEM CHILD
pokemon-black-pokemon-white
THIS IS A PROBLEM CHILD
brain-age-2-more-training-in-minutes-a-day
SUCCESSFUL
gran-turismo-3-a-spec
SUCCESSFUL
call-of-duty-modern-warfare-3
SUCCESSFUL
call-of-duty-black-ops
SUCCESSFUL
call-of-duty-black-ops-3
THIS IS A PROBLEM CHILD
call-of-duty-black-ops-ii


KeyboardInterrupt: 

[{'metascore': '\n87\n',
  'positive_critic_reviews': '64',
  'mixed_critic_reviews': '9',
  'negative_critic_reviews': '0'},
 {'metascore': '\n87\n',
  'positive_critic_reviews': '64',
  'mixed_critic_reviews': '9',
  'negative_critic_reviews': '0'},
 {'metascore': '\n87\n',
  'positive_critic_reviews': '64',
  'mixed_critic_reviews': '9',
  'negative_critic_reviews': '0'},
 {'metascore': '\n87\n',
  'positive_critic_reviews': '64',
  'mixed_critic_reviews': '9',
  'negative_critic_reviews': '0'},
 {'metascore': '\n87\n',
  'positive_critic_reviews': '64',
  'mixed_critic_reviews': '9',
  'negative_critic_reviews': '0'},
 {'metascore': '\n87\n',
  'positive_critic_reviews': '64',
  'mixed_critic_reviews': '9',
  'negative_critic_reviews': '0'}]

In [169]:
game_scores = {}
critic_reviews = []
positive_scores = my_soup.find("div",class_="distribution_wrap")
x = positive_scores.select("ol.score_counts.hover_none span.count")
for i in x:
    mylist.append(i.get_text())
game_scores["positive_critic_reviews"] = mylist[0]
game_scores["mixed_critic_reviews"]=mylist[1]
game_scores["negative_critic_reviews"]=mylist[2]
print(game_scores)

{'positive_critic_reviews': '64', 'mixed_critic_reviews': '9', 'negative_critic_reviews': '0'}
